In [ ]:
import gc 
gc.collect()

# Morlet Time Frequency Analysis - Visualizations

### Modules

In [ ]:
import mne
import pandas as pd
import numpy as np
import pickle
import os

from matplotlib import pyplot as plt
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

import math
import scipy
from scipy import stats
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Define dictionaries, subject, & conditions

In [ ]:
# set path to main folder
%cd ~/
%pwd

# path to data files
data_path = "/Volumes/Elements/data_mne/"

# subject 
subj = ['Bou_Ni'] 
sub_idx = 0 # subject number

# list of conditions
condition_list = ['produce_music', 'perceive_music_produced', 'produce_speech', 'perceive_speech_produced']
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
contrast_names = ["Music Production and Music Perception", "Speech Production and Speech Perception"]

condition_list_control = ['perceive_music_new', 'perceive_music_newrepetition', 'perceive_speech_new', 'perceive_speech_newrepetition']
contrasts_control = [["perceive_music_new", "perceive_music_newrepetition"], ["perceive_speech_new", "perceive_speech_newrepetition"]]
contrast_names_control = ["Music Perception Control and Music Perception Control Repetition", "Speech Perception Control and Speech Perception Control Repetition"]

In [ ]:
# dictionary path 
dictionary_path = data_path + subj[sub_idx] + "/dictionary/TFA"

# plot path 
plot_path = data_path + subj[sub_idx]  + "/plots/Power-Frequency-Plots/"

# freesurfer path 
fs_path = data_path + subj[sub_idx]  + '/freesurfer/Bou_Ni/elec_recon/'

### Load epoch data 

In [ ]:
# load epochs
epochs = {}

for condition in condition_list:
    preprocessed_path = data_path + subj[sub_idx] + "/preprocessed/" + condition + "/"
    print(preprocessed_path)

    for files in os.listdir(preprocessed_path):
        #filename = day + "_bipolar_epochs_preprocessed.fif"
        filename = "day1_bipolar_epochs_preprocessed.fif"

        if filename in files:
            path = preprocessed_path + files + '/'
            epochs[condition] = mne.read_epochs(path)   

In [ ]:
# channel names
picks = epochs["produce_music"].ch_names

# parameters
tmin = 0 
tmax = 300 #2.999
fmin = 1
fmax = 180
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  # log10 freq scale 

### Retrieve TFA results

In [ ]:
day = 'both'

In [ ]:
morlet_results = {}
morlet_results_day1 = {}
morlet_results_day2 = {}

for condition in condition_list:
    morlet_results[condition] = {}
    morlet_results_day1[condition] = {}
    morlet_results_day2[condition] = {}
      
    concatenated_results = {}

    file_path_day1 = f"{dictionary_path}/{condition}_day1_morlet_results.pickle"
    with open(file_path_day1, 'rb') as f:
        day1 = pickle.load(f)

    file_path_day2 = f"{dictionary_path}/{condition}_day2_morlet_results.pickle"
    with open(file_path_day2, 'rb') as f:
        day2 = pickle.load(f)

    morlet_results[condition] = np.concatenate((day1, day2))
    morlet_results_day1[condition] = day1
    morlet_results_day2[condition] = day2

In [ ]:
morlet_results_control = {}

for condition in condition_list_control:
    morlet_results_control[condition] = {}
        
    file_path_control = f"{dictionary_path}/{condition}_day1_morlet_results.pickle"
    with open(file_path_control, 'rb') as f:
        control = pickle.load(f)

    morlet_results_control[condition] = control

# Compare conditions

In [ ]:
freq_band_index = [[0,13], [13,20], [20,24], [24,33], [33,42], [42,50]]
band_names = ["Delta", "Theta", "Alpha", "Beta", "Low Gamma", "High Gamma"]

## Self Produced Speech and Music: Production - Perception 

In [ ]:
contrasts = [["produce_music", "perceive_music_produced"], ["produce_speech", "perceive_speech_produced"]]
channel_index = 42
exponent = 1

for n, contrast in enumerate(contrasts): 

    if "music" in contrast[0]: 
        color_prod = 'purple'
        color_perc = 'plum'
    elif "speech" in contrast[0]: 
        color_prod = '#008000'
        color_perc = '#90EE90'

    # Calculate mean across epochs
    data_produced = np.mean(morlet_results[contrast[0]][:, channel_index, :], axis=0) 
    data_new = np.mean(morlet_results[contrast[1]][:, channel_index, :], axis=0)

    # Calculate standard error of the mean (SEM)
    sem_produced = np.std(morlet_results[contrast[0]][:, channel_index, :], axis=0)  / np.sqrt(morlet_results[contrast[0]].shape[0])
    sem_new = np.std(morlet_results[contrast[1]][:, channel_index, :], axis=0) / np.sqrt(morlet_results[contrast[1]].shape[0])

    # Mitigate the 1/f effect by multiplying power values by frequency vector
    data_produced *= frequencies ** exponent
    data_new *= frequencies ** exponent
    sem_produced *= frequencies ** exponent
    sem_new *= frequencies ** exponent

    # Plot the Power Spectrum Density with SEM
    fig, ax = plt.subplots(figsize=(12, 3), ncols=1)

    # Plot the mean power with SEM shading for production
    ax.plot(np.arange(len(frequencies)), data_produced, color=color_prod, label='Production', linewidth=2)
    ax.fill_between(np.arange(len(frequencies)), data_produced - sem_produced, data_produced + sem_produced, color=color_prod, alpha=0.3)

    # Plot the mean power with SEM shading for perception
    ax.plot(np.arange(len(frequencies)), data_new, color=color_perc, label='Perception', linewidth=2)
    ax.fill_between(np.arange(len(frequencies)), data_new - sem_new, data_new + sem_new, color=color_perc, alpha=0.3)

    ax.set_ylabel('Power', fontsize=16)
    ax.legend(fontsize=12, loc='upper right')

    ax.set_xticks(np.arange(len(frequencies)))
    ax.set_xticklabels(np.around(frequencies, 1), rotation=90, size=12)
    ax.set_xlim(-0.5, len(frequencies) - 0.5)

    # Adding vertical lines to separate frequency bands
    for band_index in [[0, 13], [13, 20], [20, 24], [24, 33], [33, 42]]:
        ax.axvline(band_index[1] - 0.5, color="black", lw=1)

    # Adding a secondary x-axis for frequency bands
    ax2 = ax.twiny()  
    ax2.set_xlim(ax.get_xlim()) 
    band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
    ax2.set_xticks(band_midpoints)  
    ax2.set_xticklabels(band_names, rotation=0, ha='center', fontsize=12)  
    ax2.tick_params(axis='x', pad=10)  
    fig.subplots_adjust(bottom=0.2, top=0.85)

    plt.show()


In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]

for n, contrast in enumerate(contrasts): 
    data_produced = morlet_results[contrast[0]][:,channel_index,:]
    data_new = morlet_results[contrast[1]][:,channel_index,:]
    
    ratio = np.mean(((data_produced - data_new ) / data_new * 100), axis=0)

    t, p = stats.ttest_rel(data_produced, data_new)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
    t[p_corrected > 0.05] = 0
    
    fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

    im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    #im1 = ax.imshow(ratio, aspect='auto', origin='lower', interpolation='none', vmax=80, vmin=-80, cmap="seismic")
   
    ax.set_xlabel('Frequencies', size=16)
    ax.set_ylabel('Channels', size=16)
    ax.set_yticks(np.arange(t.shape[0]))
    ax.set_yticklabels(picks_H, size=12)
    ax.set_xticks(np.arange(len(frequencies)))
    ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
    ax.axhline(7.5, color="black", lw=3)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
    for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
        ax.axvline(band_index[1] - 0.5, color="black", lw=1)

    ax2 = ax.twiny()  
    ax2.set_xlim(ax.get_xlim()) 
    band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
    ax2.set_xticks(band_midpoints)  
    ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
    ax2.tick_params(axis='x', pad=10)  
    fig.subplots_adjust(bottom=0.2, top=0.85)

    ax3 = ax.twinx()  
    ax3.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax3.set_yticks(channel_midpoints)  
    ax3.set_yticklabels(brain_label, size=12, ha='left')  

    colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
    colorbar.set_label('t-statistics', size=14) 

    contrast_name = contrast[0] + "-" + contrast[1]
    #plt.savefig(plot_path + f"TFA_H-channels_t-statistics_{contrast_name}_power-frequency-plot.jpg", bbox_inches='tight')
    plt.show()


In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]

for i, morlet_results in enumerate([morlet_results_day1, morlet_results_day2]):
    for n, contrast in enumerate(contrasts): 
        data_produced = morlet_results[contrast[0]][:,channel_index,:]
        data_new = morlet_results[contrast[1]][:,channel_index,:]
        
        t, p = stats.ttest_rel(data_produced, data_new)
        p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
        t[p_corrected > 0.05] = 0
        
        fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

        im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    
        ax.set_xlabel('Frequencies', size=16)
        ax.set_ylabel('Channels', size=16)
        ax.set_yticks(np.arange(t.shape[0]))
        ax.set_yticklabels(picks_H, size=12)
        ax.set_xticks(np.arange(len(frequencies)))
        ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
        ax.axhline(7.5, color="black", lw=3)
        for channel_pos_index in [[0,5], [5,7], [7,11]]: 
            ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
        for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
            ax.axvline(band_index[1] - 0.5, color="black", lw=1)

        ax2 = ax.twiny()  
        ax2.set_xlim(ax.get_xlim()) 
        band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
        ax2.set_xticks(band_midpoints)  
        ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
        ax2.tick_params(axis='x', pad=10)  
        fig.subplots_adjust(bottom=0.2, top=0.85)

        ax3 = ax.twinx()  
        ax3.set_ylim(ax.get_ylim()) 
        channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
        ax3.set_yticks(channel_midpoints)  
        ax3.set_yticklabels(brain_label, size=12, ha='left')  

        colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
        colorbar.set_label('t-statistics', size=14) 

        contrast_name = contrast[0] + "-" + contrast[1]
        #plt.savefig(plot_path + f"TFA_H-channels_t-statistics_{contrast_name}_{day}_power-frequency-plot.jpg", bbox_inches='tight')
        plt.show()


### Linear mixed effect models


#### Model for music

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  
epochs=np.r_[0:100]
data = []
for day, morlet_results in zip(['Day1', 'Day2'], [morlet_results_day1, morlet_results_day2]):
    for condition in ['produce_music', 'perceive_music_produced']:
        for pick, channel_idx in zip(picks_H, channel_index):
            for freq_idx, freq in enumerate(frequencies):
                for epoch_idx in epochs:
                    result = morlet_results[condition][epoch_idx, channel_idx, freq_idx]
                    data.append({
                        'Subject': 'BouNi', 
                        'Condition': condition, 
                        'Day': day, 
                        'Channel': pick, 
                        'Frequency': freq, 
                        'Epoch': epoch_idx, 
                        'Score': result
                    })

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['produce_music', 'perceive_music_produced'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ C(Condition)", data_music, groups=data_music["Subject"])
result = model.fit()
print("music")
print(result.summary())

In [ ]:
# Extract and print the coefficients with higher precision
print("Model Coefficients (unrounded):")
params = result.params
for param in params.index:
    print(f"{param}: {params[param]:.3e}")

# Extract and print the standard errors
print("\nStandard Errors (unrounded):")
bse = result.bse
for param in bse.index:
    print(f"{param}: {bse[param]:.3e}")

# Extract and print p-values
print("\nP-values (unrounded):")
pvalues = result.pvalues
for param in pvalues.index:
    print(f"{param}: {pvalues[param]:.3e}")

# Extract and print confidence intervals
print("\nConfidence Intervals (unrounded):")
conf_int = result.conf_int()
for param in conf_int.index:
    print(f"{param}: [{conf_int.loc[param][0]:.3e}, {conf_int.loc[param][1]:.3e}]")

#### Model for speech 

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num = 50, base = 10)  
epochs=np.r_[0:100]
data = []
for day, morlet_results in zip(['Day1', 'Day2'], [morlet_results_day1, morlet_results_day2]):
    for condition in ['produce_speech', 'perceive_speech_produced']:
        for pick, channel_idx in zip(picks_H, channel_index):
            for freq_idx, freq in enumerate(frequencies):
                for epoch_idx in epochs:
                    result = morlet_results[condition][epoch_idx, channel_idx, freq_idx]
                    data.append({
                        'Subject': 'BouNi', 
                        'Condition': condition, 
                        'Day': day, 
                        'Channel': pick, 
                        'Frequency': freq, 
                        'Epoch': epoch_idx, 
                        'Score': result
                    })

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['produce_speech', 'perceive_speech_produced'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ C(Condition)", data_speech, groups=data_speech["Subject"])
result = model.fit()
print("speech")
print(result.summary().as_text())

In [ ]:
# Extract and print the coefficients with higher precision
print("Model Coefficients (unrounded):")
params = result.params
for param in params.index:
    print(f"{param}: {params[param]:.3e}")

# Extract and print the standard errors
print("\nStandard Errors (unrounded):")
bse = result.bse
for param in bse.index:
    print(f"{param}: {bse[param]:.3e}")

# Extract and print p-values
print("\nP-values (unrounded):")
pvalues = result.pvalues
for param in pvalues.index:
    print(f"{param}: {pvalues[param]:.3e}")

# Extract and print confidence intervals
print("\nConfidence Intervals (unrounded):")
conf_int = result.conf_int()
for param in conf_int.index:
    print(f"{param}: [{conf_int.loc[param][0]:.3e}, {conf_int.loc[param][1]:.3e}]")

## Speech and Music Repetition: First Hearing - Second Hearing 

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
brain_label = ["Right Primary \n Auditory Cortex", "Right Associative \n Auditory Cortex", "Left Primary \n Auditory Cortex", "Left Associative \n Auditory Cortex"]
channel_position = [[0,5], [6,7], [8,11], [11,len(picks_H)]]

for n, contrast in enumerate(contrasts_control): 
    data_firsthearing = morlet_results_control[contrast[0]][:,channel_index,:]
    data_secondhearing = morlet_results_control[contrast[1]][:,channel_index,:]
    
    ratio = np.mean(((data_firsthearing - data_secondhearing ) / data_secondhearing * 100), axis=0)

    t, p = stats.ttest_rel(data_firsthearing, data_secondhearing)
    p_corrected = multipletests(p.flatten(), method='fdr_bh')[1].reshape(p.shape)
    t[p_corrected > 0.05] = 0
    
    fig, ax = plt.subplots(figsize=(12, 8), ncols=1)

    im1 = ax.imshow(t, aspect='auto', origin='lower', vmin=-10, vmax=10, interpolation='none',  cmap='seismic')
    ax.set_xlabel('Frequencies', size=16)
    ax.set_ylabel('Channels', size=16)
    ax.set_yticks(np.arange(t.shape[0]))
    ax.set_yticklabels(picks_H, size=12)
    ax.set_xticks(np.arange(len(frequencies)))
    ax.set_xticklabels((np.around(frequencies,1)), rotation=90, size=12)
    ax.axhline(7.5, color="black", lw=3)
    for channel_pos_index in [[0,5], [5,7], [7,11]]: 
        ax.axhline(channel_pos_index[1] + 0.5, color="black", lw=1.5)
    for band_index in [[0,13], [13,20], [20,24], [24,33], [33,42]]:
        ax.axvline(band_index[1] - 0.5, color="black", lw=1)

    ax2 = ax.twiny()  
    ax2.set_xlim(ax.get_xlim()) 
    band_midpoints = [(index[0] + index[1]) / 2 for index in freq_band_index]
    ax2.set_xticks(band_midpoints)  
    ax2.set_xticklabels(band_names, rotation=0, ha='center', size=12)  
    ax2.tick_params(axis='x', pad=10)  
    fig.subplots_adjust(bottom=0.2, top=0.85)

    ax3 = ax.twinx()  
    ax3.set_ylim(ax.get_ylim()) 
    channel_midpoints = [(index[0] + index[1]) / 2 for index in channel_position]
    ax3.set_yticks(channel_midpoints)  
    ax3.set_yticklabels(brain_label, size=12, ha='left')  

    colorbar = fig.colorbar(im1, ax=ax, fraction=0.05, pad=0.17)
    colorbar.set_label('t-statistics', size=14) 

    contrast_name = contrast[0] + "-" + contrast[1]
    plt.show()


### Linear mixed effect models

#### Model for music 

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num=50, base=10)
epochs = np.r_[0:100]

data = []

for condition in ['perceive_music_new', 'perceive_music_newrepetition']:
    for pick, channel_idx in zip(picks_H, channel_index):
        for freq_idx, freq in enumerate(frequencies):
            for epoch_idx in epochs:
                result = morlet_results_control[condition][epoch_idx, channel_idx, freq_idx]
                data.append({
                    'Subject': 'BouNi', 
                    'Condition': condition, 
                    'Channel': pick, 
                    'Frequency': freq, 
                    'Epoch': epoch_idx, 
                    'Score': result
                })

data_music = pd.DataFrame(data)

data_music['Condition'] = pd.Categorical(data_music['Condition'], 
                                          categories=['perceive_music_new', 'perceive_music_newrepetition'],
                                          ordered=True)


In [ ]:
model = smf.mixedlm("Score ~ Condition ", data_music, groups=data_music["Subject"])
result = model.fit()
print("speech")
print(result.summary())

In [ ]:
# Extract and print the coefficients with higher precision
print("Model Coefficients (unrounded):")
params = result.params
for param in params.index:
    print(f"{param}: {params[param]:.3e}")

# Extract and print the standard errors
print("\nStandard Errors (unrounded):")
bse = result.bse
for param in bse.index:
    print(f"{param}: {bse[param]:.3e}")

# Extract and print p-values
print("\nP-values (unrounded):")
pvalues = result.pvalues
for param in pvalues.index:
    print(f"{param}: {pvalues[param]:.3e}")

# Extract and print confidence intervals
print("\nConfidence Intervals (unrounded):")
conf_int = result.conf_int()
for param in conf_int.index:
    print(f"{param}: [{conf_int.loc[param][0]:.3e}, {conf_int.loc[param][1]:.3e}]")

#### Model for speech 

In [ ]:
channel_index = np.r_[42:50, 89:96]
picks_H = [picks[index] for index in channel_index]
frequencies = np.logspace(np.log10(fmin), np.log10(fmax), num=50, base=10)
epochs = np.r_[0:100]

data = []

for condition in ['perceive_speech_new', 'perceive_speech_newrepetition']:
    print(condition)
    for pick, channel_idx in zip(picks_H, channel_index):
        for freq_idx, freq in enumerate(frequencies):
            for epoch_idx in epochs:
                result = morlet_results_control[condition][epoch_idx, channel_idx, freq_idx]
                data.append({
                    'Subject': 'BouNi', 
                    'Condition': condition, 
                    'Channel': pick, 
                    'Frequency': freq, 
                    'Epoch': epoch_idx, 
                    'Score': result
                })

data_speech = pd.DataFrame(data)

data_speech['Condition'] = pd.Categorical(data_speech['Condition'], 
                                          categories=['perceive_speech_new', 'perceive_speech_newrepetition'],
                                          ordered=True)

In [ ]:
model = smf.mixedlm("Score ~ Condition", data_speech, groups=data_speech["Subject"])
result = model.fit()
print("speech")
print(result.summary())

In [ ]:
# Extract and print the coefficients with higher precision
print("Model Coefficients (unrounded):")
params = result.params
for param in params.index:
    print(f"{param}: {params[param]:.3e}")

# Extract and print the standard errors
print("\nStandard Errors (unrounded):")
bse = result.bse
for param in bse.index:
    print(f"{param}: {bse[param]:.3e}")

# Extract and print p-values
print("\nP-values (unrounded):")
pvalues = result.pvalues
for param in pvalues.index:
    print(f"{param}: {pvalues[param]:.3e}")

# Extract and print confidence intervals
print("\nConfidence Intervals (unrounded):")
conf_int = result.conf_int()
for param in conf_int.index:
    print(f"{param}: [{conf_int.loc[param][0]:.3e}, {conf_int.loc[param][1]:.3e}]")